### Import

In [147]:
import numpy as np
import sklearn
import pandas as pd
import matplotlib
import seaborn as sns

# 각 라이브러리 버전 출력
print("Numpy version:", np.__version__)
print("Scikit-learn version:", sklearn.__version__)
print("Pandas version:", pd.__version__)
print("Matplotlib version:", matplotlib.__version__)
print("Seaborn version:", sns.__version__)

Numpy version: 1.26.4
Scikit-learn version: 1.5.1
Pandas version: 1.5.3
Matplotlib version: 3.9.2
Seaborn version: 0.13.2


In [148]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import  OrdinalEncoder
from sklearn.ensemble import ExtraTreesClassifier

### Data Load

In [149]:
train = pd.read_csv('../../train.csv').drop(columns=['ID'])
test = pd.read_csv('../../test.csv').drop(columns=['ID'])

In [150]:
X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

### Data Pre-processing

In [151]:
categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부"
]

In [152]:
# 카테고리형 컬럼들을 문자열로 변환
for col in categorical_columns:
    X[col] = X[col].astype(str)
    test[col] = test[col].astype(str)

In [153]:
# 결측값을 채울 칼럼 목록
columns_to_fill = [
    '단일 배아 이식 여부', '착상 전 유전 검사 사용 여부', '착상 전 유전 진단 사용 여부',
    '총 생성 배아 수', '미세주입된 난자 수', '미세주입에서 생성된 배아 수', '이식된 배아 수',
    '미세주입 배아 이식 수', '저장된 배아 수', '미세주입 후 저장된 배아 수', '해동된 배아 수',
    '해동 난자 수', '수집된 신선 난자 수', '저장된 신선 난자 수', '혼합된 난자 수',
    '대리모 여부'
]

# 시술 유형이 'DI'인 경우에만 결측값을 0으로 채우기
X.loc[X['시술 유형'] == 'DI', columns_to_fill] = X.loc[X['시술 유형'] == 'DI', columns_to_fill].fillna(0)

# 1. '난자 출처'의 결측값을 '본인 제공'으로 채우기
X['난자 출처'].replace('알 수 없음','본인 제공', inplace=True)

# 2. '난자 기증자 나이' 결측값을 시술 당시 나이로 채우기
X.loc[X['난자 기증자 나이'] == '알 수 없음', '난자 기증자 나이'] = X['시술 당시 나이']

# 3. '파트너 정자와 혼합된 난자 수'의 결측값을 0으로 채우기
X['파트너 정자와 혼합된 난자 수'].fillna(0, inplace=True)

# 4. '기증자 정자와 혼합된 난자 수'의 결측값을 1로 채우기
X['기증자 정자와 혼합된 난자 수'].fillna(1, inplace=True)

# 5. '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부'의 결측값을 0으로 채우기
for column in ['동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부']:
    X[column].fillna(0, inplace=True)

    # 변경 내용 확인
print(X[['난자 출처', '난자 기증자 나이', '파트너 정자와 혼합된 난자 수', 
            '기증자 정자와 혼합된 난자 수', '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부']].head())


   난자 출처 난자 기증자 나이  파트너 정자와 혼합된 난자 수  기증자 정자와 혼합된 난자 수 동결 배아 사용 여부  \
0  본인 제공   만18-34세               5.0               0.0         0.0   
1  본인 제공   만45-50세               1.0               0.0         0.0   
2  본인 제공   만18-34세               7.0               0.0         0.0   
3  본인 제공   만35-37세               4.0               0.0         0.0   
4  본인 제공   만18-34세               6.0               0.0         0.0   

  신선 배아 사용 여부 기증 배아 사용 여부  
0         1.0         0.0  
1         1.0         0.0  
2         1.0         0.0  
3         1.0         0.0  
4         1.0         0.0  


In [154]:
# 각 열의 결측값 개수 확인
missing_values_count = X.isnull().sum()

# 결측값이 있는 열만 필터링
missing_columns = missing_values_count[missing_values_count > 0]

print("결측값이 있는 열과 개수:")
print(missing_columns)

결측값이 있는 열과 개수:
임신 시도 또는 마지막 임신 경과 연수    246981
난자 채취 경과일                 57488
난자 해동 경과일                254915
난자 혼합 경과일                 53735
배아 이식 경과일                 43566
배아 해동 경과일                215982
dtype: int64


In [155]:

X['난자 채취 경과일'].fillna(1, inplace=True)
X['난자 해동 경과일'].fillna(0, inplace=True)
X['난자 혼합 경과일'].fillna(0, inplace=True)
X['배아 이식 경과일'].fillna(0, inplace=True)
X['배아 해동 경과일'].fillna(0, inplace=True)

In [156]:
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X_train_encoded = X.copy()
X_train_encoded[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns])

X_test_encoded = test.copy()
X_test_encoded[categorical_columns] = ordinal_encoder.transform(test[categorical_columns])

In [157]:
columns_to_drop = [
        "임신 시도 또는 마지막 임신 경과 연수",
        "배아 생성 주요 이유",
        "PGD 시술 여부",
        "PGS 시술 여부"
]
X_train_encoded = X_train_encoded.drop(columns = columns_to_drop)    
X_test_encoded = X_test_encoded.drop(columns = columns_to_drop)  

In [158]:
# 병합할 칼럼들
columns_to_merge = [
    '난자 채취 경과일', '난자 해동 경과일', '난자 혼합 경과일',
    '배아 이식 경과일', '배아 해동 경과일'
]

# 새로운 칼럼 생성: 각 경과일의 합
X_train_encoded['총 경과일'] = X_train_encoded[columns_to_merge].sum(axis=1)

# 기존 칼럼 제거
X_train_encoded = X_train_encoded.drop(columns=columns_to_merge)

# 새로운 칼럼 생성: 각 경과일의 합
X_test_encoded['총 경과일'] = X_test_encoded[columns_to_merge].sum(axis=1)

# 기존 칼럼 제거
X_test_encoded = X_test_encoded.drop(columns=columns_to_merge)

In [159]:
# 각 열의 결측값 개수 확인
missing_values_count = X_train_encoded.isnull().sum()

# 결측값이 있는 열만 필터링
missing_columns = missing_values_count[missing_values_count > 0]

print("결측값이 있는 열과 개수:")
print(missing_columns)


결측값이 있는 열과 개수:
Series([], dtype: int64)


In [160]:
X_train_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256351 entries, 0 to 256350
Data columns (total 59 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   시술 시기 코드            256351 non-null  float64
 1   시술 당시 나이            256351 non-null  float64
 2   시술 유형               256351 non-null  float64
 3   특정 시술 유형            256351 non-null  float64
 4   배란 자극 여부            256351 non-null  float64
 5   배란 유도 유형            256351 non-null  float64
 6   단일 배아 이식 여부         256351 non-null  float64
 7   착상 전 유전 검사 사용 여부    256351 non-null  float64
 8   착상 전 유전 진단 사용 여부    256351 non-null  float64
 9   남성 주 불임 원인          256351 non-null  float64
 10  남성 부 불임 원인          256351 non-null  float64
 11  여성 주 불임 원인          256351 non-null  float64
 12  여성 부 불임 원인          256351 non-null  float64
 13  부부 주 불임 원인          256351 non-null  float64
 14  부부 부 불임 원인          256351 non-null  float64
 15  불명확 불임 원인           256351 non-nul

### Train

In [161]:
model = ExtraTreesClassifier(random_state=42)

model.fit(X_train_encoded, y)

ExtraTreesClassifier(random_state=42)

**앙상블 및 Optuna**

In [162]:
import optuna
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score
import numpy as np

In [163]:
def optimize_xgboost(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 5)
    }
    model = XGBClassifier(**params, random_state=42, use_label_encoder=False, eval_metric="logloss")
    scores = cross_val_score(model, X_train_encoded, y, cv=5, scoring="roc_auc")
    return np.mean(scores)

In [164]:
def optimize_lightgbm(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 10, 50),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0)
    }
    model = LGBMClassifier(**params, random_state=42)
    scores = cross_val_score(model, X_train_encoded, y, cv=5, scoring="roc_auc")  
    return np.mean(scores)

In [165]:
def optimize_random_forest(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 300),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_samples_split": trial.suggest_int("min_samples_split", 3, 10),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 4)
    }
    model = RandomForestClassifier(**params, random_state=42)
    scores = cross_val_score(model, X_train_encoded, y, cv=5, scoring="roc_auc")  
    return np.mean(scores)


In [166]:
# Optuna 스터디 실행
print("Optimizing XGBoost...")
xgb_study = optuna.create_study(direction="maximize")
xgb_study.optimize(optimize_xgboost, n_trials=30)
best_xgb_params = xgb_study.best_params

print("Optimizing LightGBM...")
lgbm_study = optuna.create_study(direction="maximize")
lgbm_study.optimize(optimize_lightgbm, n_trials=30)
best_lgbm_params = lgbm_study.best_params

print("Optimizing RandomForest...")
rf_study = optuna.create_study(direction="maximize")
rf_study.optimize(optimize_random_forest, n_trials=30)
best_rf_params = rf_study.best_params

[I 2025-02-12 16:05:12,747] A new study created in memory with name: no-name-a8d496af-a131-4898-9d3d-8c57535e9136


Optimizing XGBoost...


c:\Users\hoyeong\anaconda3\envs\lg_ai\lib\site-packages\xgboost\core.py:158: UserWarning: [16:05:12] WARNING: D:\bld\xgboost-split_1722454234704\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\hoyeong\anaconda3\envs\lg_ai\lib\site-packages\xgboost\core.py:158: UserWarning: [16:05:15] WARNING: D:\bld\xgboost-split_1722454234704\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\hoyeong\anaconda3\envs\lg_ai\lib\site-packages\xgboost\core.py:158: UserWarning: [16:05:18] WARNING: D:\bld\xgboost-split_1722454234704\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\hoyeong\anaconda3\envs\lg_ai\lib\site-packages\xgboost\core.py:158: UserWarning: [16:05:20] WARNING: D:\bld\xgboost-split_1722454234704\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warni

Optimizing LightGBM...
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 52982, number of negative: 152098
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009935 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 674
[LightGBM] [Info] Number of data points in the train set: 205080, number of used features: 56
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258348 -> initscore=-1.054573
[LightGBM] [Info] Start training from score -1.054573
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 52983, number of negative: 152098
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010486 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not eno

[I 2025-02-12 16:09:34,741] Trial 0 finished with value: 0.7350735488929596 and parameters: {'n_estimators': 281, 'max_depth': 12, 'learning_rate': 0.28229356517204657, 'num_leaves': 19, 'subsample': 0.7310297907963179}. Best is trial 0 with value: 0.7350735488929596.


...

[I 2025-02-12 16:11:48,952] Trial 28 finished with value: 0.7383514800075563 and parameters: {'n_estimators': 136, 'max_depth': 8, 'learning_rate': 0.03728850552827087, 'num_leaves': 25, 'subsample': 0.7176575936654896}. Best is trial 7 with value: 0.7390278742230043.


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 52982, number of negative: 152098
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008620 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 674
[LightGBM] [Info] Number of data points in the train set: 205080, number of used features: 56
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258348 -> initscore=-1.054573
[LightGBM] [Info] Start training from score -1.054573
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be

[I 2025-02-12 16:11:52,216] Trial 29 finished with value: 0.73897333159684 and parameters: {'n_estimators': 198, 'max_depth': 3, 'learning_rate': 0.12074034989975403, 'num_leaves': 20, 'subsample': 0.7690475120010147}. Best is trial 7 with value: 0.7390278742230043.
[I 2025-02-12 16:11:52,217] A new study created in memory with name: no-name-36cd3c52-ad35-449d-aabc-f2d5603600fd


Optimizing RandomForest...


[I 2025-02-12 16:12:52,752] Trial 0 finished with value: 0.7312616254537615 and parameters: {'n_estimators': 133, 'max_depth': 8, 'min_samples_split': 9, 'min_samples_leaf': 1}. Best is trial 0 with value: 0.7312616254537615.
[I 2025-02-12 16:14:35,390] Trial 1 finished with value: 0.7312786362803084 and parameters: {'n_estimators': 225, 'max_depth': 8, 'min_samples_split': 8, 'min_samples_leaf': 1}. Best is trial 1 with value: 0.7312786362803084.
[I 2025-02-12 16:16:30,924] Trial 2 finished with value: 0.7337800798931781 and parameters: {'n_estimators': 218, 'max_depth': 10, 'min_samples_split': 10, 'min_samples_leaf': 4}. Best is trial 2 with value: 0.7337800798931781.
[I 2025-02-12 16:18:15,858] Trial 3 finished with value: 0.7276588504834886 and parameters: {'n_estimators': 292, 'max_depth': 6, 'min_samples_split': 5, 'min_samples_leaf': 4}. Best is trial 2 with value: 0.7337800798931781.
[I 2025-02-12 16:18:59,352] Trial 4 finished with value: 0.7295839652561995 and parameters: {'

In [167]:
# 최적화된 모델 생성
xgb_model = XGBClassifier(**best_xgb_params, random_state=42, use_label_encoder=False, eval_metric="logloss")
lgbm_model = LGBMClassifier(**best_lgbm_params, random_state=42)
rf_model = RandomForestClassifier(**best_rf_params, random_state=42)

In [168]:
# XGBoost 모델의 파라미터 확인
print("XGBoost Best Parameters:")
print(xgb_model.get_params())

# LightGBM 모델의 파라미터 확인
print("\nLightGBM Best Parameters:")
print(lgbm_model.get_params())

# RandomForest 모델의 파라미터 확인
print("\nRandomForest Best Parameters:")
print(rf_model.get_params())

XGBoost Best Parameters:
{'objective': 'binary:logistic', 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': 0.8144971423877854, 'device': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': 'logloss', 'feature_types': None, 'gamma': 1.7007844464984736, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': 0.02916793537022215, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': 5, 'max_leaves': None, 'min_child_weight': None, 'missing': nan, 'monotone_constraints': None, 'multi_strategy': None, 'n_estimators': 499, 'n_jobs': None, 'num_parallel_tree': None, 'random_state': 42, 'reg_alpha': None, 'reg_lambda': None, 'sampling_method': None, 'scale_pos_weight': None, 'subsample': 0.7171684798315661, 'tree_method': None, 'validate_parameters': None, 'verbosity': None, 'use_label_encoder'

In [169]:
# Soft Voting 앙상블
ensemble_model = VotingClassifier(
    estimators=[
        ("xgb", xgb_model),
        ("lgbm", lgbm_model),
        ("rf", rf_model)
    ],
    voting="soft"
)

In [170]:
# 전체 데이터로 학습
ensemble_model.fit(X_train_encoded, y)

c:\Users\hoyeong\anaconda3\envs\lg_ai\lib\site-packages\xgboost\core.py:158: UserWarning: [17:11:14] WARNING: D:\bld\xgboost-split_1722454234704\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 66228, number of negative: 190123
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.012564 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 677
[LightGBM] [Info] Number of data points in the train set: 256351, number of used features: 56
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258349 -> initscore=-1.054568
[LightGBM] [Info] Start training from score -1.054568
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be

VotingClassifier(estimators=[('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.8144971423877854,
                                            device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric='logloss',
                                            feature_types=None,
                                            gamma=1.7007844464984736,
                                            grow_policy=None,
                                            importance_type=None,
                                            interac...
                                            n_estimators=499, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=42, ...)),
                             ('lgbm',
                              LGBMClassifier(learning_rate=0.08960125620945708,
                                             max_depth=3, n_estimators=282,
                                             num_leaves=19, random_state=42,
                                             subsample=0.624359030746388)),
                             ('rf',
                              RandomForestClassifier(max_depth=12,
                                                     min_samples_leaf=2,
                                                     min_samples_split=5,
                                                     n_estimators=251,
                                                     random_state=42))],
                 voting='soft')

### Predict

In [171]:
pred_proba = ensemble_model.predict_proba(X_test_encoded)[:, 1]

### Submission

In [172]:
sample_submission = pd.read_csv('../../sample_submission.csv')
sample_submission['probability'] = pred_proba

In [173]:
sample_submission.to_csv('./baseline_submit.csv', index=False)

In [174]:
pred_proba

array([0.00414168, 0.00172599, 0.09605177, ..., 0.47767794, 0.20950989,
       0.00442333])

In [175]:
unique, counts = np.unique(sample_submission['probability'], return_counts=True)
print(unique, counts)

[1.98709193e-04 2.20525121e-04 2.27300112e-04 ... 6.09636958e-01
 6.18985393e-01 6.21986261e-01] [1 1 1 ... 1 1 1]
